#### ライブラリ

In [15]:
from __future__ import division
import os
import io
import re
import sys
import pyaudio
import librosa
from google.cloud import speech
from google.cloud import storage
import numpy as np
import pandas as pd
from six.moves import queue
from natsort import natsorted
import IPython
from scipy.io import wavfile
import soundfile as sf
import scipy.signal
import matplotlib.pyplot as plt
#import noisereduce as nr
%matplotlib inline

#### 検証する文章

*田中さん、脈拍をはかりますね。あ、腕まくりはいりませんよ。ちょっと失礼しますね。じゃあ付けますね。...よいしょっと。では測ります。...今日はいつもより速めですね。...93ですね。お疲れ様でした。*

#### 懸念点
* 周囲の雑音による精度低下
* 数字の読み上げ途中に間があった場合の誤登録（はちじゅう...ろくですね）

#### 検証に使う音声

6つの音声ファイルを文字起こしした際の精度を比較する。

* 静かな環境・間なし（File1）
* ややうるさい環境・間なし（File2）
* うるさい環境・間なし（File3）
* 静かな環境・間あり（File4）
* ややうるさい環境・間あり（File5）
* うるさい環境・間あり（File6）

検証文はAudacityで録音、.wav形式で保存したのちフリーの環境音声と合成。
また、環境音がある音声に関しては`noisereduce`ライブラリを用いたノイズ軽減処理を行った場合の精度も確認する。

In [5]:
IPython.display.display(IPython.display.Audio(url="../google_speech_to_text_API/data/test_nopause.wav"))
IPython.display.display(IPython.display.Audio(url="../google_speech_to_text_API/data/test_nopause_noisy.wav"))
IPython.display.display(IPython.display.Audio(url="../google_speech_to_text_API/data/test_nopause_verynoisy.wav"))
IPython.display.display(IPython.display.Audio(url="../google_speech_to_text_API/data/test_pause.wav"))
IPython.display.display(IPython.display.Audio(url="../google_speech_to_text_API/data/test_pause_noisy.wav"))
IPython.display.display(IPython.display.Audio(url="../google_speech_to_text_API/data/test_pause_verynoisy.wav"))

#### Google Speech-to-Text APIで文字起こし

In [10]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../google_speech_to_text_API/onyx-glazing-302315-6269164c75cd.json"

In [24]:
client = storage.Client()
bucket = client.get_bucket('mahos_stt_bucket')
blobs = bucket.list_blobs()
filter_dir = 'pause'
blob_list = [blob.name for blob in blobs if filter_dir in blob.name ]

print(blob_list)

['test_nopause.wav', 'test_nopause_noisy.wav', 'test_nopause_verynoisy.wav', 'test_pause.wav', 'test_pause_noisy.wav', 'test_pause_verynoisy.wav']


In [25]:
path_list =[]
for x in blob_list:
    name = "gs://mahos_stt_bucket/{}".format(x)
    path_list.append(name)
print(path_list)

['gs://mahos_stt_bucket/test_nopause.wav', 'gs://mahos_stt_bucket/test_nopause_noisy.wav', 'gs://mahos_stt_bucket/test_nopause_verynoisy.wav', 'gs://mahos_stt_bucket/test_pause.wav', 'gs://mahos_stt_bucket/test_pause_noisy.wav', 'gs://mahos_stt_bucket/test_pause_verynoisy.wav']


In [22]:
def sample_long_running_recognize(gcs_uri):
    """
    Print start and end time of each word spoken in audio file from Cloud Storage

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech.SpeechClient()
    # When enabled, the first result returned by the API will include a list
    # of words and the start and end time offsets (timestamps) for those words.
    
    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        enable_word_time_offsets=True,
        audio_channel_count=1,
        language_code="ja-JP",
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print(u"{}の文字起こし結果".format(gcs_uri.split('/')[-1]))
    response = operation.result()
    
    
    stt_result = {"文字起こし":[]}
    confidence = {"Confidenceスコア":[]}
    time_count = {"単語":[],"Start time":[],"End time":[]}

    for result in response.results:
        alternative = result.alternatives[0]
        stt_result["文字起こし"].append(alternative.transcript)
        print(u"文字起こし: {}".format(alternative.transcript))
        confidence["Confidenceスコア"].append(alternative.confidence)
        for word in alternative.words:
            time_count["単語"].append(word.word)
            start_time = word.start_time
            end_time= word.end_time
            
            time_count["Start time"].append(start_time)
            time_count["End time"].append(end_time)
            
    return stt_result,confidence,time_count

In [ ]:
for gcs_uri in path_list:
    stt_result, confidence, time_count = sample_long_running_recognize(gcs_uri)

test_nopause.wavの文字起こし結果
文字起こし: 田中さん脈拍を測りますねは腕まくりは要りませんよちょっと失礼しますねじゃあつけますね良いちょっとでは計ります今日はいつもより早めですね
文字起こし: 93ですねお疲れ様でした
test_nopause_noisy.wavの文字起こし結果
文字起こし: 田中さん脈拍を測りますねあ腕まくりは要りませんよちょっと失礼しますねじゃあつけますね良いショットでは計ります今日はいつもより早めですね93ですねお疲れ様でした
test_nopause_verynoisy.wavの文字起こし結果
文字起こし: ちょっとしたらそうだねじゃあちょっとで分かります今日はいつもより早いですね93ですねお疲れ様でした
test_pause.wavの文字起こし結果
文字起こし: 田中さん脈拍を測りますね
文字起こし: 腕まくりは要りませんよ
文字起こし: ちょっと失礼しますね
文字起こし: じゃあつけますね
文字起こし: よいしょっと
文字起こし: では計ります
文字起こし: 今日はいつもより早めですね
文字起こし: 90
文字起こし: 3ですねお疲れ様でした
test_pause_noisy.wavの文字起こし結果
文字起こし: 田中さん脈拍を測りますねあ腕まくりは要りませんよちょっと失礼しますねじゃあつけますね良いショットではかかります今日はいつもより早めですね93ですねお疲れ様でした
test_pause_verynoisy.wavの文字起こし結果
